In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

graph = tf.Graph()
with graph.as_default():
    z = tf.Variable(5, name="z")
    
f = x*x*y + y + 2


In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [4]:
z.graph == tf.get_default_graph()

False

In [5]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15


10
15


In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [13]:
scaled_housing_data_plus_bias = housing_data_plus_bias / housing_data_plus_bias.max()

In [29]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()


Epoch 0 MSE = 5.7452083
Epoch 100 MSE = 5.7169795
Epoch 200 MSE = 5.689039
Epoch 300 MSE = 5.661392
Epoch 400 MSE = 5.634029
Epoch 500 MSE = 5.606951
Epoch 600 MSE = 5.5801525
Epoch 700 MSE = 5.553631
Epoch 800 MSE = 5.5273876
Epoch 900 MSE = 5.5014186


In [47]:
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={X: X_batch, y: y_batch}))
        for batch in range(n_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size
            X_batch = scaled_housing_data_plus_bias[start:end,:]
            y_batch = housing.target.reshape(-1,1)[start:end,:]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()


Epoch 0 MSE = 0.67457104
Epoch 100 MSE = 0.47240567
Epoch 200 MSE = 0.8065054
Epoch 300 MSE = 0.9646932
Epoch 400 MSE = 1.0219002
Epoch 500 MSE = 1.0400836
Epoch 600 MSE = 1.0444746
Epoch 700 MSE = 1.0441253
Epoch 800 MSE = 1.0422112
Epoch 900 MSE = 1.0397781


In [34]:
print(housing.target.reshape(-1,1).shape)
housing.target.shape

(20640, 1)


(20640,)